In [1]:
import json
import numpy as np
from tqdm import tqdm
from pprint import pprint
from sklearn.cluster import KMeans

from label_processor import CStdLib

In [2]:
with open('archived/result_logits.json', encoding='utf-8') as f:
    logits_json = json.load(f)

In [3]:
embed = CStdLib(single=False)

In [4]:
pbar = tqdm(logits_json.items())
i = 0
for key, prod_dict in pbar:
    optional_tags = prod_dict['optional_tags']
    num_class = len(optional_tags)
    imgs_tags = prod_dict['imgs_tags']
    imgs_logits = []
    for i in range(len(imgs_tags)):
        imgs_logits.append(np.fromstring(imgs_tags[i][key + '_{}.jpg'.format(i)], dtype=np.float32, sep=' '))
    imgs_logits = np.array(imgs_logits)
    cluster = KMeans(n_clusters=num_class, random_state=42).fit(imgs_logits)
    labels = cluster.labels_
    match_score = np.zeros((num_class, num_class))
    # match_score[i, j] 表示 kmeans 分类结果为 i 和 optional_tags[j] 的匹配度
    embeded_tags = [embed(tag) for tag in optional_tags]
    not_matched = False
    for i in range(num_class):
        id_i = np.where(labels == i)[0]
        logits = imgs_logits[id_i]
        for j in range(num_class):
            logits_ids = embeded_tags[j]
            if len(logits_ids) > 0:
                logits_j = logits[:, logits_ids]
                match_score[i, j] = np.mean(logits_j)
            else:
                not_matched = True
                match_score[i, j] = 1
    match_score_norm = match_score / np.linalg.norm(match_score, axis=0, ord=3)
    # print(key, '\n', match_score_norm)
    label_id = np.argmax(match_score_norm, axis=1)
    # print(label_id)
    for i in range(num_class):
        id_i = np.where(labels == i)[0]
        for j in id_i:
            logits_json[key]['imgs_tags'][j][key + '_{}.jpg'.format(j)] = optional_tags[label_id[i]]
    if not_matched:
        print(key, '\n', match_score, '\n', match_score_norm)
        pprint(logits_json[key])

  5%|▌         | 284/5331 [00:03<00:57, 87.86it/s] 

624952669722 
 [[3.54169917 1.        ]
 [6.12443686 1.        ]] 
 [[0.54519422 0.79370053]
 [0.94276995 0.79370053]]
{'imgs_tags': [{'624952669722_0.jpg': '珊瑚色组合'},
               {'624952669722_1.jpg': '珊瑚色组合'},
               {'624952669722_2.jpg': '橄榄色组合'},
               {'624952669722_3.jpg': '橄榄色组合'},
               {'624952669722_4.jpg': '橄榄色组合'}],
 'optional_tags': ['珊瑚色组合', '橄榄色组合']}


  6%|▋         | 341/5331 [00:03<00:48, 103.89it/s]

614043736149 
 [[ 6.78489113  5.1024971   2.56172848  1.          4.02346802]
 [ 2.91709399  2.64869952  9.82762909  1.          3.01735854]
 [ 2.72273445  9.69181824  3.23731184  1.          2.28493643]
 [ 3.47204447  4.16564274  2.93793297  1.         11.85724735]
 [11.45318031  4.11650181  3.28069067  1.          4.18891811]] 
 [[0.54765214 0.47965296 0.25119923 0.58480355 0.32804042]
 [0.23545739 0.24898722 0.96368249 0.58480355 0.24601054]
 [0.21976938 0.91106555 0.31744592 0.58480355 0.18629488]
 [0.28025101 0.3915853  0.28808928 0.58480355 0.96674221]
 [0.92445974 0.38696588 0.32169958 0.58480355 0.34152986]]
{'imgs_tags': [{'614043736149_0.jpg': '条纹'},
               {'614043736149_1.jpg': '桔色'},
               {'614043736149_2.jpg': '红色'},
               {'614043736149_3.jpg': '桔色'},
               {'614043736149_4.jpg': '红色'},
               {'614043736149_5.jpg': '黑色'},
               {'614043736149_6.jpg': '条纹'},
               {'614043736149_7.jpg': '黑色'},
               {

 15%|█▌        | 815/5331 [00:08<00:44, 102.15it/s]

...

 85%|████████▌ | 4537/5331 [00:45<00:07, 105.33it/s]

577541680180 
 [[13.46714783  1.          6.60329819  1.92246294]
 [ 3.42867184  1.          2.63183498 11.72202206]
 [ 2.86667657  1.         10.48777962  2.60973549]
 [ 3.29647803  1.          3.67912197  8.85310936]] 
 [[0.98675411 0.62996052 0.57563284 0.14502435]
 [0.25122291 0.62996052 0.22942636 0.88427118]
 [0.21004484 0.62996052 0.91425681 0.19686995]
 [0.24153691 0.62996052 0.32072206 0.66784975]]
{'imgs_tags': [{'577541680180_0.jpg': '红色'},
               {'577541680180_1.jpg': '红色'},
               {'577541680180_2.jpg': '蓝色'},
               {'577541680180_3.jpg': '蓝色'},
               {'577541680180_4.jpg': '橙色'},
               {'577541680180_5.jpg': '蓝色'},
               {'577541680180_6.jpg': '橙色'},
               {'577541680180_7.jpg': '蓝色'},
               {'577541680180_8.jpg': '蓝色'}],
 'optional_tags': ['橙色', '深色', '红色', '蓝色']}


 90%|████████▉ | 4783/5331 [00:48<00:06, 90.21it/s] 

612126042525 
 [[ 1.         11.84072876]
 [ 1.         11.24886417]] 
 [[0.79370053 0.81351158]
 [0.79370053 0.77284781]]
{'imgs_tags': [{'612126042525_0.jpg': '957黑色'},
               {'612126042525_1.jpg': '957黑色'},
               {'612126042525_2.jpg': '957黑色'},
               {'612126042525_3.jpg': '6802格子'},
               {'612126042525_4.jpg': '957黑色'},
               {'612126042525_5.jpg': '957黑色'},
               {'612126042525_6.jpg': '957黑色'}],
 'optional_tags': ['6802格子', '957黑色']}


100%|██████████| 5331/5331 [00:54<00:00, 98.41it/s] 


In [5]:
with open('./result_labels.json', 'w') as f:
    json.dump(logits_json, f, indent=4, ensure_ascii=False)